Now there are many things that we could do here.

1) the overfit is a problem. We could go back and clean up our feature set. There might be some gains to be made by getting rid of the noise.

2) We could also see how removing the nulls but including dietary information performs. Though its a slight change to the question we could still possibly get some improvements there.

3) We could take our regression problem and turn it into a classifier. 

4) We could make it simpler still by instead of classifying on each possible value, group reviews to some decided high and low values.

In [21]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Epicurious - Recipes with Rating and Nutrition

In [22]:
raw_data = pd.read_csv('Desktop/Thinkful/epi_r.csv')
raw_data.head(2)

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
list(raw_data.columns)

['title',
 'rating',
 'calories',
 'protein',
 'fat',
 'sodium',
 '#cakeweek',
 '#wasteless',
 '22-minute meals',
 '3-ingredient recipes',
 '30 days of groceries',
 'advance prep required',
 'alabama',
 'alaska',
 'alcoholic',
 'almond',
 'amaretto',
 'anchovy',
 'anise',
 'anniversary',
 'anthony bourdain',
 'aperitif',
 'appetizer',
 'apple',
 'apple juice',
 'apricot',
 'arizona',
 'artichoke',
 'arugula',
 'asian pear',
 'asparagus',
 'aspen',
 'atlanta',
 'australia',
 'avocado',
 'back to school',
 'backyard bbq',
 'bacon',
 'bake',
 'banana',
 'barley',
 'basil',
 'bass',
 'bastille day',
 'bean',
 'beef',
 'beef rib',
 'beef shank',
 'beef tenderloin',
 'beer',
 'beet',
 'bell pepper',
 'berry',
 'beverly hills',
 'birthday',
 'biscuit',
 'bitters',
 'blackberry',
 'blender',
 'blue cheese',
 'blueberry',
 'boil',
 'bok choy',
 'bon appétit',
 'bon app��tit',
 'boston',
 'bourbon',
 'braise',
 'bran',
 'brandy',
 'bread',
 'breadcrumbs',
 'breakfast',
 'brie',
 'brine',
 'brisk

In [24]:
raw_data.rating.describe()

count    20052.000000
mean         3.714467
std          1.340829
min          0.000000
25%          3.750000
50%          4.375000
75%          4.375000
max          5.000000
Name: rating, dtype: float64

In [25]:
(raw_data.isnull().sum()*100/len(raw_data)).sort_values(ascending = False)

fat                   20.860762
protein               20.756034
sodium                20.541592
calories              20.531618
turkey                 0.000000
fourth of july         0.000000
fritter                0.000000
frittata               0.000000
friendsgiving          0.000000
freezer food           0.000000
freeze/chill           0.000000
frankenrecipe          0.000000
frangelico             0.000000
france                 0.000000
fontina                0.000000
fortified wine         0.000000
food processor         0.000000
fruit                  0.000000
florida                0.000000
flat bread             0.000000
flaming hot summer     0.000000
fish                   0.000000
fig                    0.000000
feta                   0.000000
frozen dessert         0.000000
fry                    0.000000
fruit juice            0.000000
father's day           0.000000
grappa                 0.000000
grapefruit             0.000000
                        ...    
orzo    

In [26]:
raw_data.corr().loc[:,"rating"].sort_values(ascending=False)

rating                  1.000000
bon appétit             0.124549
peanut free             0.096087
soy free                0.092965
tree nut free           0.081347
bake                    0.076978
roast                   0.073499
fall                    0.067343
sauté                   0.066553
dinner                  0.060957
kosher                  0.059071
winter                  0.055126
pescatarian             0.054034
thanksgiving            0.052990
onion                   0.049997
grill/barbecue          0.048081
high fiber              0.047827
gourmet                 0.047354
no sugar added          0.047197
tomato                  0.046579
quick & easy            0.045216
herb                    0.044253
pork                    0.042463
beef                    0.041550
cheese                  0.041215
low carb                0.040738
mixer                   0.040620
christmas               0.040595
sugar conscious         0.040573
braise                  0.040155
          

It doesn't look like removing the nulls but including dietary information will increase correlation with our target. So we will just drop the columns.

In [29]:
raw_data = raw_data.drop(columns = ['fat', 'sodium', 'calories', 'protein'], axis = 1)

In [33]:
raw_data['high'] = np.where(raw_data['rating'] >= 4, 1, 0)

In [43]:
all_features = raw_data.corr().loc[:,"rating"].sort_values(ascending=False)
features = all_features.iloc[1:30].index.tolist()

In [44]:
features

['high',
 'bon appétit',
 'peanut free',
 'soy free',
 'tree nut free',
 'bake',
 'roast',
 'fall',
 'sauté',
 'dinner',
 'kosher',
 'winter',
 'pescatarian',
 'thanksgiving',
 'onion',
 'grill/barbecue',
 'high fiber',
 'gourmet',
 'no sugar added',
 'tomato',
 'quick & easy',
 'herb',
 'pork',
 'beef',
 'cheese',
 'low carb',
 'mixer',
 'christmas',
 'sugar conscious']

In [51]:
Y = raw_data['high']
X = raw_data.drop(['rating', 'title', 'high'], 1)


In [54]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score


In [ ]:
svc = SVC()
svc.fit(X,Y)
svc.score(X, Y)
cross_val_score(svc, X, Y, cv=5)

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
